In [2]:
import os
import pandas as pd
import json
import glob
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import  classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [3]:
city_list=os.listdir("C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather")
file_path="C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather\\"
weather_list=[]
for p in city_list:
    json_list=os.listdir(file_path+p)
    s=json_list.index('2016-1.json')
    actual_list=json_list[s:]
    for i in actual_list:
        fp=open(file_path+p+"\\"+i,"r")
        d=fp.read()
        data=json.loads(d)
        feature_list=['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph',         'tempF','WindChillF','humidity','time']
        
        for i in data["data"]["weather"]:
            for j in range(24):
                weather_per_hr=dict()
                for k in feature_list:
                    weather_per_hr[k]=i["hourly"][j][k]
                weather_per_hr['date']=i['date']
                weather_per_hr['airport']=data["data"]["request"][0]["query"]
                weather_list.append(weather_per_hr)
        fp.close()
df_weather=pd.DataFrame(weather_list)

df_weather.to_csv("weather_features.csv")

In [4]:
df_weather["air"]=""

In [5]:
df_weather["air"]=[x[0:3] for x in df_weather["airport"]]

In [6]:
df_weather["int_time"]=0

In [7]:
df_weather["int_time"]=[int(x) for x in df_weather["time"]]


In [8]:
df_weather.shape

(263160, 17)

In [9]:
df_weather

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport,air,int_time
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,0,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,0
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,100,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,100
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,200,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,200
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,300,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,300
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,400,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263155,26,296,113,0.0,10,1014,0,57,27,67,67,70,1900,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,1900
263156,26,298,113,0.0,10,1014,0,58,27,67,67,71,2000,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2000
263157,26,300,113,0.0,10,1014,0,58,27,67,67,73,2100,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2100
263158,23,302,113,0.0,10,1014,0,58,25,66,66,74,2200,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2200


In [10]:
files=glob.glob('C:\\Users\\Abi\\Desktop\\ML_project\\Flight Data-20200805T074725Z-001\\Flight Data\**\*',recursive=True)
df_flight=pd.DataFrame()
for i in files:
    try:
        i.index(".csv")
    except:
        continue
    d=pd.read_csv(i)
    d=d[['FlightDate','Quarter','Year','Month'
    ,'DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15',           'ArrDelayMinutes','Origin','Dest']]
    df_flight=pd.concat([df_flight,d])

    

In [11]:
airports=['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
df_flight=df_flight[df_flight["Dest"].isin(airports)]
df_flight=df_flight[df_flight["Origin"].isin(airports)]





In [12]:
df_flight=df_flight.dropna()

In [13]:
df_flight["round_time"]=0

In [14]:
#to calculate the round-off value for the departure time
temp=[]
for x in df_flight["CRSDepTime"]: 
    if(x%100<30):
        temp.append((int(x/100)*100)%2400)
    else:
        temp.append(((int(x/100)+1)*100)%2400)
df_flight["round_time"]=temp

In [15]:
df_flight.shape

(1851436, 18)

In [16]:
df_flight

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,round_time
57,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,SEA,JFK,800
58,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,12478,1613.0,1602,0.0,11.0,SEA,JFK,800
59,2016-01-03,1,2016,1,3,743.0,0.0,745,0.0,14747,12478,1547.0,1602,0.0,0.0,SEA,JFK,800
60,2016-01-04,1,2016,1,4,737.0,0.0,745,0.0,14747,12478,1551.0,1602,0.0,0.0,SEA,JFK,800
61,2016-01-05,1,2016,1,5,708.0,0.0,710,0.0,14747,12478,1524.0,1527,0.0,0.0,SEA,JFK,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458684,2017-09-25,3,2017,9,25,2038.0,0.0,2045,0.0,14771,12478,449.0,522,0.0,0.0,SFO,JFK,2100
458687,2017-09-25,3,2017,9,25,1200.0,0.0,1205,0.0,11618,13204,1428.0,1449,0.0,0.0,EWR,MCO,1200
458688,2017-09-25,3,2017,9,25,1143.0,0.0,1141,2.0,13204,11618,1409.0,1417,0.0,0.0,MCO,EWR,1200
458695,2017-09-25,3,2017,9,25,2157.0,0.0,2159,0.0,12889,12478,540.0,557,0.0,0.0,LAS,JFK,2200


In [17]:
#merging weather dataframe and flight dataframe
result=pd.merge(left=df_weather,right=df_flight,left_on=["date","int_time","air"],right_on=["FlightDate","round_time","Origin"])

In [18]:
result=result.dropna()

In [19]:
target=result["ArrDel15"]

In [20]:
features=result.copy()
features=features.drop(["ArrDel15"],axis=1)

In [21]:
features=features.drop(["date","airport","air","FlightDate","Origin","Dest","ArrDelayMinutes"],axis=1)

In [22]:
feature_selt=pd.DataFrame(SelectKBest(f_classif,k=20).fit_transform(features,target))

In [23]:
#feature set
train_feature=feature_selt.head(int(0.8*len(features)))
test_feature=feature_selt.tail(int(0.2*len(features)))

In [24]:
#target set
train_target=target.head(int(0.8*len(target)))
test_target=target.tail(int(0.2*len(features)))

In [25]:
#logistic regression classification
LogReg=LogisticRegression()
LogReg.fit(train_feature,train_target)
logit_predict=LogReg.predict(test_feature)

In [26]:
print(classification_report(test_target,logit_predict))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    291769
         1.0       0.91      0.66      0.77     78518

    accuracy                           0.92    370287
   macro avg       0.91      0.82      0.86    370287
weighted avg       0.92      0.92      0.91    370287



In [27]:
 #decision tree classifier
 dtc=DecisionTreeClassifier()
 dtc=dtc.fit(train_feature,train_target)
 tree_pred=dtc.predict(test_feature)
 

In [28]:
print(classification_report(test_target,tree_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97    291769
         1.0       0.90      0.89      0.89     78518

    accuracy                           0.96    370287
   macro avg       0.94      0.93      0.93    370287
weighted avg       0.95      0.96      0.96    370287



In [29]:
#xgboost
xgb = GradientBoostingClassifier(random_state=0)
xgb.fit(train_feature,train_target)

GradientBoostingClassifier(random_state=0)

In [30]:
xgb_pred=xgb.predict(test_feature)

In [31]:
print(classification_report(test_target,xgb_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95    291769
         1.0       0.93      0.69      0.79     78518

    accuracy                           0.92    370287
   macro avg       0.92      0.84      0.87    370287
weighted avg       0.92      0.92      0.92    370287



In [32]:
#extra tree classifier
etc=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc.fit(train_feature,train_target)
extratree_pred=etc.predict(test_feature)


In [33]:

print(classification_report(test_target,extratree_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.98      0.95    291769
         1.0       0.91      0.71      0.80     78518

    accuracy                           0.92    370287
   macro avg       0.92      0.85      0.88    370287
weighted avg       0.92      0.92      0.92    370287



In [34]:
#oversampling method
#os = oversampling
os=SMOTE(random_state=2)
train_feature_os,train_target_os= os.fit_sample(train_feature,train_target)

In [35]:
#logistic reg
LogReg_os=LogisticRegression()
LogReg_os.fit(train_feature_os,train_target_os)
logit_predict_os=LogReg_os.predict(test_feature)

In [36]:
print(classification_report(test_target,logit_predict_os))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94    291769
         1.0       0.77      0.77      0.77     78518

    accuracy                           0.90    370287
   macro avg       0.85      0.85      0.85    370287
weighted avg       0.90      0.90      0.90    370287



In [37]:
#decision tree classifier
dtc_os=DecisionTreeClassifier()
dtc_os.fit(train_feature_os,train_target_os)
tree_pred_os=dtc_os.predict(test_feature)

In [38]:
print(classification_report(test_target,tree_pred_os))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95    291769
         1.0       0.81      0.82      0.82     78518

    accuracy                           0.92    370287
   macro avg       0.88      0.89      0.88    370287
weighted avg       0.92      0.92      0.92    370287



In [39]:
#xgboost
xgb_os = GradientBoostingClassifier(random_state=0)
xgb_os.fit(train_feature_os,train_target_os)
xgb_pred_os=xgb_os.predict(test_feature)

In [40]:
print(classification_report(test_target,xgb_pred_os))

              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95    291769
         1.0       0.86      0.72      0.78     78518

    accuracy                           0.91    370287
   macro avg       0.89      0.84      0.86    370287
weighted avg       0.91      0.91      0.91    370287



In [41]:
#extra tree classifier
etc_os=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc_os.fit(train_feature_os,train_target_os)
extratree_pred_os=etc_os.predict(test_feature)


In [42]:
print(classification_report(test_target,extratree_pred_os))

              precision    recall  f1-score   support

         0.0       0.94      0.97      0.95    291769
         1.0       0.86      0.76      0.81     78518

    accuracy                           0.92    370287
   macro avg       0.90      0.86      0.88    370287
weighted avg       0.92      0.92      0.92    370287



In [43]:
#under-sampling
#us- under sampling
us =NearMiss()
train_feature_us,train_target_us= us.fit_sample(train_feature,train_target)

In [44]:
#logistic reg
LogReg_us=LogisticRegression()
LogReg_us.fit(train_feature_us,train_target_us)
logit_predict_us=LogReg_us.predict(test_feature)

In [45]:
print(classification_report(test_target,logit_predict_us))


              precision    recall  f1-score   support

         0.0       0.93      0.87      0.90    291769
         1.0       0.61      0.76      0.68     78518

    accuracy                           0.85    370287
   macro avg       0.77      0.82      0.79    370287
weighted avg       0.86      0.85      0.85    370287



In [46]:
#decision tree classifier
dtc_us=DecisionTreeClassifier()
dtc_us.fit(train_feature_us,train_target_us)
tree_pred_us=dtc_us.predict(test_feature)

In [47]:
print(classification_report(test_target,tree_pred_us))

              precision    recall  f1-score   support

         0.0       0.97      0.85      0.91    291769
         1.0       0.62      0.90      0.73     78518

    accuracy                           0.86    370287
   macro avg       0.79      0.88      0.82    370287
weighted avg       0.90      0.86      0.87    370287



In [48]:
#xgboost
xgb_us = GradientBoostingClassifier(random_state=0)
xgb_us.fit(train_feature_us,train_target_us)
xgb_pred_us=xgb_us.predict(test_feature)

In [49]:
print(classification_report(test_target,xgb_pred_us))

              precision    recall  f1-score   support

         0.0       0.93      0.85      0.89    291769
         1.0       0.59      0.78      0.67     78518

    accuracy                           0.84    370287
   macro avg       0.76      0.82      0.78    370287
weighted avg       0.86      0.84      0.84    370287



In [50]:
#extra tree classifier
etc_us=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc_us.fit(train_feature_us,train_target_us)
extratree_pred_us=etc_us.predict(test_feature)

In [51]:
print(classification_report(test_target,extratree_pred_us))

              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86    291769
         1.0       0.51      0.81      0.62     78518

    accuracy                           0.79    370287
   macro avg       0.72      0.80      0.74    370287
weighted avg       0.85      0.79      0.81    370287

